In [ ]:
# PyTorch 설치
!pip install torch torchvision

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# CIFAR-10 데이터셋 다운로드 및 변환
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform
)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=4, shuffle=True, num_workers=2
)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform
)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=4, shuffle=False, num_workers=2
)

classes = (
    'plane', 'car', 'bird', 'cat',
    'deer', 'dog', 'frog', 'horse', 'ship', 'truck'
)

# Two-Layer Neural Network 정의
class TwoLayerNN(nn.Module):
    def __init__(self):
        super(TwoLayerNN, self).__init__()
        self.fc1 = nn.Linear(3 * 32 * 32, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 3 * 32 * 32)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# 모델, 손실 함수, 최적화기 초기화
net = TwoLayerNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# 모델 학습
for epoch in range(10):  # 데이터셋을 여러 번 반복합니다.
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        # 변화도(gradient) 매개변수를 0으로 초기화
        optimizer.zero_grad()

        # 순전파 + 역전파 + 최적화
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 통계 출력
        running_loss += loss.item()
        if i % 2000 == 1999:  # 미니배치마다 출력
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('학습 종료')

# 모델 저장
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

# 학습된 모델을 시험 데이터셋에서 테스트
net = TwoLayerNN()
net.load_state_dict(torch.load(PATH))

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'네트워크의 정확도: {100 * correct / total:.2f}%')


하이퍼파라미터를 조정해보세요. learning rate, batch Size 수정 등을 통해 정확도를 높일 수 있습니다.


학습률을 높이면 빠르게 수렴하지만 발산할 수 있고, 낮추면 안정적으로 학습할 수 있지만 학습 시간이 길어질 수 있습니다.
더 큰 미니배치 크기를 사용하면 학습 속도가 빨라질 수 있지만 메모리 사용량이 늘어날 수 있습니다. 작은 미니배치 크기를 사용하면 모델이 더 안정적으로 학습될 수 있지만 학습 시간이 길어질 수 있습니다.

정규화 기법인 배치 정규화(Batch Normalization) 또는 드롭아웃(Dropout)을 사용하여 과적합을 줄일 수 있습니다.

다양한 옵티마이저 시도하여 학습을 시도할 수 있습니다. SGD, Adam, RMSprop 등을 시도해보세요.
